In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from tqdm import tqdm

In [12]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
HIST_LEN = 50
FUTURE_LEN = 60
TRAIN_RATIO = 0.9   
BATCH_SIZE = 64
EPOCHS = 15
LR = 1e-4
SEED = 42
SOCIAL_GRID_SIZE = 64
SOCIAL_FEATURE_SIZE = 128
torch.manual_seed(SEED)
np.random.seed(SEED)

In [ ]:
# === DATASET ===
class TrajectoryDataset(Dataset):
    def __init__(self, data, train=True, pos_mean=None, pos_std=None, vel_mean=None, vel_std=None):
        self.data = data
        self.train = train
        self.pos_mean = pos_mean
        self.pos_std = pos_std
        self.vel_mean = vel_mean
        self.vel_std = vel_std

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        scene = self.data[idx]  
        # Extract features for all agents
        positions = scene[:, :HIST_LEN, :2]  # (50, T, 2)
        velocities = scene[:, :HIST_LEN, 2:4]  # (50, T, 2)
        headings = scene[:, :HIST_LEN, 4]  # (50, T)
        
        # Normalize positions and velocities
        if self.pos_mean is not None and self.pos_std is not None:
            pos_mask = ~(positions == 0).all(axis=2, keepdims=True)
            positions = (positions - self.pos_mean) / (self.pos_std + 1e-6)
            positions = positions * pos_mask
        
        if self.vel_mean is not None and self.vel_std is not None:
            vel_mask = ~(velocities == 0).all(axis=2, keepdims=True)
            velocities = (velocities - self.vel_mean) / (self.vel_std + 1e-6)
            velocities = velocities * vel_mask
            
        # Convert heading to sine/cosine
        sin_head = np.sin(headings)[..., None]  # (50, T, 1)
        cos_head = np.cos(headings)[..., None]  # (50, T, 1)
    
        # Combine features
        hist = np.concatenate([
            positions,
            velocities,
            sin_head,
            cos_head
        ], axis=-1)  # (50, T, 2+2+1+1+4=10)
        
        if self.train:
            # Get ego's future trajectory
            future = scene[0, HIST_LEN:HIST_LEN+FUTURE_LEN, :2]
            if self.pos_mean is not None and self.pos_std is not None:
                future_mask = ~(future == 0).all(axis=1, keepdims=True)
                future = (future - self.pos_mean) / (self.pos_std + 1e-6)
                future = future * future_mask
                
            return hist.astype(np.float32), future.astype(np.float32)
        
        return hist.astype(np.float32)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=64, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, num_layers=num_layers)

    def forward(self, x):  
        B, A, T, D = x.shape
        x = x.view(B * A, T, D)
        _, (h, _) = self.lstm(x)
        h = h[-1]  
        return h.view(B, A, -1)  

class Decoder(nn.Module):

    def __init__(self, hidden_dim=128, out_dim=2):
        super().__init__()
        self.lstm = nn.LSTM(hidden_dim, 64, batch_first=True)
        self.fc = nn.Linear(64, out_dim)

    def forward(self, h):
        h = h.unsqueeze(1).repeat(1, FUTURE_LEN, 1)  
        out, _ = self.lstm(h)
        return self.fc(out)


class SocialPooling(nn.Module):
    def __init__(self, hidden_dim=64, out_dim=128):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, out_dim),
            nn.ReLU(),
            nn.Linear(out_dim, out_dim)
        )

    def forward(self, h, positions=None):
        social_feat = h.mean(dim=1)  
        return self.fc(social_feat)  

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.pooling = SocialPooling()
        self.decoder = Decoder(hidden_dim=SOCIAL_FEATURE_SIZE)

    def forward(self, x):
        encoded = self.encoder(x)  # (B, A, H)
        last_pos = x[:, :, -1, :]  # (B, A, 2)
        pooled = self.pooling(encoded, last_pos)  # grid-based pooling
        pred = self.decoder(pooled)  # (B, 60, 2)
        return pred

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(FUTURE_LEN * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, traj):
        return self.model(traj.view(traj.size(0), -1))

In [ ]:
# === TRAINING ===
def train_model(generator, discriminator, train_loader, val_loader, pos_mean, pos_std, epochs=10, lr=1e-3):
    g_opt = torch.optim.Adam(generator.parameters(), lr=lr)
    d_opt = torch.optim.Adam(discriminator.parameters(), lr=lr)
    mse_loss = nn.MSELoss()
    bce_loss = nn.BCELoss()

    for epoch in range(epochs):
        generator.train()
        discriminator.train()
        total_g_loss = 0
        total_d_loss = 0

        for hist, future in tqdm(train_loader, desc=f"[Epoch {epoch+1}] Training"):
            hist, future = hist.to(DEVICE), future.to(DEVICE)

            # === Train Discriminator ===
            real_out = discriminator(future)
            fake_traj = generator(hist).detach()
            fake_out = discriminator(fake_traj)

            real_label = torch.ones_like(real_out)
            fake_label = torch.zeros_like(fake_out)

            d_loss = bce_loss(real_out, real_label) + bce_loss(fake_out, fake_label)
            d_opt.zero_grad()
            d_loss.backward()
            d_opt.step()

            # === Train Generator ===
            fake_traj = generator(hist)
            pred_out = discriminator(fake_traj)

            adv_loss = bce_loss(pred_out, torch.ones_like(pred_out))
            #l2_loss = mse_loss(fake_traj, future)
            pred_real = fake_traj * (pos_std + 1e-6) + pos_mean
            future_real = future * (pos_std + 1e-6) + pos_mean
            l2_loss = mse_loss(pred_real, future_real)
            g_loss = adv_loss + l2_loss

            g_opt.zero_grad()
            g_loss.backward()
            g_opt.step()

            total_g_loss += l2_loss.item()
            total_d_loss += d_loss.item()

        avg_train_mse = total_g_loss / len(train_loader)

        # === Validation ===
        generator.eval()
        val_loss = 0
        with torch.no_grad():
            for hist, future in val_loader:
                hist, future = hist.to(DEVICE), future.to(DEVICE)
                pred = generator(hist)
                pred_real = pred * (pos_std + 1e-6) + pos_mean
                future_real = future * (pos_std + 1e-6) + pos_mean
                val_loss += mse_loss(pred_real, future_real).item()

        avg_val_mse = val_loss / len(val_loader)
        print(f"Epoch {epoch+1}: Train MSE = {avg_train_mse:.4f}, Val MSE = {avg_val_mse:.4f}, D Loss = {total_d_loss / len(train_loader):.4f}")


In [6]:
# === INFERENCE ===
def predict(generator, dataloader):
    generator.eval()
    all_preds = []
    with torch.no_grad():
        for hist in tqdm(dataloader, desc="Predicting"):
            hist = hist.to(DEVICE)
            pred = generator(hist)
            all_preds.append(pred.cpu().numpy())

    return np.concatenate(all_preds, axis=0)

In [ ]:
# === MAIN SCRIPT ===
def main():
    # === Load Data ===
    train_npz = np.load("/kaggle/input/cse-251-b-2025/train.npz")
    test_npz = np.load("/kaggle/input/cse-251-b-2025/test_input.npz")

    train_data = train_npz['data']  # shape: (10000, 50, 110, 6)
    test_data = test_npz['data']    # shape: (2100, 50, 50, 6)

    positions = train_data[..., :2].reshape(-1, 2)
    mask = ~(positions == 0).all(axis=1)  # ignore padded zeros
    pos_mean = positions[mask].mean(axis=0)
    pos_std = positions[mask].std(axis=0)

    velocities = train_data[..., 2:4].reshape(-1, 2)
    vel_mask = ~(velocities == 0).all(axis=1)  # ignore padded zeros
    vel_mean = velocities[vel_mask].mean(axis=0)
    vel_std = velocities[vel_mask].std(axis=0)


    # === Train/Val Split ===
    full_dataset = TrajectoryDataset(train_data, train=True, pos_mean=pos_mean, pos_std=pos_std, 
                                     vel_mean=vel_mean, vel_std=vel_std)
    train_size = int(TRAIN_RATIO * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_set, val_set = random_split(full_dataset, [train_size, val_size])

    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

    test_set = TrajectoryDataset(test_data, train=False, pos_mean=pos_mean, pos_std=pos_std)
    test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

    # === Initialize Model ===
    generator = Generator().to(DEVICE)
    discriminator = Discriminator().to(DEVICE)
    #print("Initialize Model done")

    # === Train ===
    train_model(generator, discriminator, train_loader, val_loader, epochs=EPOCHS, lr=LR, 
                pos_mean=torch.from_numpy(pos_mean).float(),
                pos_std=torch.from_numpy(pos_std).float())

    # === Predict ===
    preds = predict(generator, test_loader)  # shape: (2100, 60, 2)
    preds = preds * (pos_std + 1e-6) + pos_mean

    # === Generate Submission ===
    ids = np.repeat(np.arange(len(test_set)), FUTURE_LEN)
    x = preds[:, :, 0].reshape(-1)
    y = preds[:, :, 1].reshape(-1)

    output_df = pd.DataFrame({'x': x, 'y': y})
    output_df.index.name = 'index'
    output_df.to_csv('social_gan_3.csv')
    print("Saved gan submission3.csv")

if __name__ == "__main__":
    main()

[Epoch 1] Training: 100%|██████████| 141/141 [02:45<00:00,  1.18s/it]


Epoch 1: Train MSE = 5734515.2004, Val MSE = 2381630.1328, D Loss = 1.1942


[Epoch 2] Training: 100%|██████████| 141/141 [02:49<00:00,  1.20s/it]


Epoch 2: Train MSE = 2207936.6933, Val MSE = 1752255.6719, D Loss = 0.9646


[Epoch 3] Training: 100%|██████████| 141/141 [02:49<00:00,  1.20s/it]


...

Epoch 14: Train MSE = 131325.7334, Val MSE = 126682.2583, D Loss = 0.1405


[Epoch 15] Training: 100%|██████████| 141/141 [02:43<00:00,  1.16s/it]


Epoch 15: Train MSE = 118939.8271, Val MSE = 110168.9370, D Loss = 0.1369


Predicting: 100%|██████████| 33/33 [00:08<00:00,  4.08it/s]


Saved gan submission3.csv
